## import modules and setup csv to be read

In [ ]:
import pandas as pd
import os
import numpy as np

csvpath = os.path.join('..','Resources','purchase_data.csv')

hero =  pd.read_csv(csvpath)

## player count

In [ ]:
## code snippit found online to clean the header names(remove spaces, lowercase, ect)
hero.columns = hero.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
## count of unique players
x = hero.sn.unique()
total_players = [len(x)]
# output to table
pd.DataFrame(total_players,columns=["Total Players"])


## purchasing analysis (total)

In [ ]:
## keeping of variable full count
fullcount=len(x)

## output list intialization that will be appended to
y=[]
x=len(hero.item_id.unique())

## add unique item total to list
y.append(x)
## add average price to list
x=hero["price"].mean()
z='${:.2f}'.format(x)
y.append(z)
## add number of purchases to list
x=len(hero.index)
y.append(x)
## add total rev to list
x=hero["price"].sum()
z='${:.2f}'.format(x)
y.append(z)
## add list y to data frame 'output' and print
outputPA=pd.DataFrame(columns=["Number of Unique Items","Average Price","Number of Purchases","Total Revenue"])
outputPA.loc[0] = y
outputPA


## gender demographics

In [ ]:
## output list intialization that will be appended to
male = []
female = []
other = []

## group by gender, then isolate only the males
gender = hero.groupby('gender')
Mgender=gender.get_group('Male')
## count the number of unique names in males, append to male list
male.append(len(Mgender.sn.unique()))
x='{:.2%}'.format(len(Mgender.sn.unique())/fullcount)
male.append(x)

## group by gender, then isolate only the females
gender = hero.groupby('gender')
Fgender=gender.get_group('Female')
## count the number of unique names in females, append to female list
female.append(len(Fgender.sn.unique()))
x='{:.2%}'.format(len(Fgender.sn.unique())/fullcount)
female.append(x)

## group by gender, then isolate only the others
gender = hero.groupby('gender')
Ogender=gender.get_group('Other / Non-Disclosed')
## count the number of unique names in others, append to others list
other.append(len(Ogender.sn.unique()))
x='{:.2%}'.format(len(Ogender.sn.unique())/fullcount)
other.append(x)

## add lists to output dataframe
outputGD=pd.DataFrame(columns=["Total Count","Percentage of Players"])
outputGD.loc[0] = male
outputGD.loc[1] = female
outputGD.loc[2] = other
outputGD

## purchasing analysis (gender)